In [5]:
%%writefile bfs_dfs.cu
#include <iostream>
#include <vector>
#include <queue>
#include <stack>

using namespace std;

// Function to perform Breadth-First Search (BFS) using CUDA for parallel execution
__global__ void BFS_CUDA(int* adj, int* visited, int V, int start) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;

    if (tid == start) {
        visited[tid] = 1;
        printf("%d ", tid);
    }

    __syncthreads();

    if (visited[tid]) {
        for (int i = 0; i < V; ++i) {
            if (adj[tid * V + i] && !visited[i]) {
                visited[i] = 1;
                printf("%d ", i);
            }
        }
    }
}

// Function to perform Depth-First Search (DFS) using CUDA for parallel execution
__global__ void DFS_CUDA(int* adj, int* visited, int V, int start) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    stack<int> st;

    if (tid == start) {
        visited[tid] = 1;
        printf("%d ", tid);
        st.push(tid);
    }

    __syncthreads();

    while (!st.empty()) {
        int u = st.top();
        st.pop();

        for (int i = 0; i < V; ++i) {
            if (adj[u * V + i] && !visited[i]) {
                visited[i] = 1;
                printf("%d ", i);
                st.push(i);
            }
        }
    }
}

int main() {
    int V = 5; // Number of vertices
    int E = 5; // Number of edges (assuming a dense graph)
    vector<int> adj(V * V, 0); // Adjacency matrix

    // Adding edges
    adj[0 * V + 1] = 1;
    adj[0 * V + 2] = 1;
    adj[1 * V + 3] = 1;
    adj[1 * V + 4] = 1;

    // Device memory allocation
    int* d_adj;
    int* d_visited;
    cudaMalloc(&d_adj, V * V * sizeof(int));
    cudaMalloc(&d_visited, V * sizeof(int));

    // Copying data from host to device
    cudaMemcpy(d_adj, &adj[0], V * V * sizeof(int), cudaMemcpyHostToDevice);

    cout << "BFS traversal: ";
    BFS_CUDA<<<1, V>>>(d_adj, d_visited, V, 0); // Starting BFS from vertex 0
    cout << endl;

    cout << "DFS traversal: ";
    DFS_CUDA<<<1, V>>>(d_adj, d_visited, V, 0); // Starting DFS from vertex 0
    cout << endl;

    return 0;
}


Overwriting bfs_dfs.cu


In [6]:
%%script bash
nvcc bfs_dfs.cu -o assign1

bfs_dfs.cu(58): warning #177-D: variable "E" was declared but never referenced
      int E = 5;
          ^

Remark: The warnings can be suppressed with "-diag-suppress <warning-number>"

bfs_dfs.cu(32): error: calling a __host__ function("std::stack<int,     ::std::deque<int, ::std::allocator<int> > > ::stack<    ::std::deque<int, ::std::allocator<int> > , void> ()") from a __global__ function("DFS_CUDA") is not allowed

bfs_dfs.cu(32): error: identifier "std::stack<int,     ::std::deque<int, ::std::allocator<int> > > ::stack<    ::std::deque<int, ::std::allocator<int> > , void> " is undefined in device code

bfs_dfs.cu(37): error: calling a __host__ function("std::stack<int,     ::std::deque<int, ::std::allocator<int> > > ::push(const int &)") from a __global__ function("DFS_CUDA") is not allowed

bfs_dfs.cu(37): error: identifier "std::stack<int,     ::std::deque<int, ::std::allocator<int> > > ::push" is undefined in device code

bfs_dfs.cu(42): error: calling a __host__ function("s

CalledProcessError: Command 'b'nvcc bfs_dfs.cu -o assign1\n'' returned non-zero exit status 1.

In [ ]:
!./assign1

Sequential Bubble Sort Time: 1e-06 seconds
Parallel Bubble Sort Time: 8e-06 seconds
Parallel Merge Sort Time: 0 seconds
